In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
import os
from pathlib import Path

# 如果出现未能成功切换到指定目录的情况，请检查设置的目录是否存在或准确
new_version_path = Path(os.getenv('VNPY2.0.8'))
sys.path.insert(0, str(new_version_path))
print(sys.path)

import vnpy
print(vnpy.__version__)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

from datetime import datetime
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
# from boll_channel_strategy import BollChannelStrategy
# from turtle_signal_strategy import TurtleSignalStrategy
from utility import (
    vt_trade_to_df,
    load_data,
    get_output_path,
    trade_zh_to_en,
    strategy_dict,
    comodity_to_vt_symbol,
    continuous_backtest
)

### 运行回测

In [ ]:
commodity = 'rb'
data_mode = 'main'
interval = '1h'
strategy_name = 'turtle'
strategy_params = {}
capital = 100000
start = datetime(2018, 1, 1)
end = datetime(2019, 12, 1)

trade_df, pnl_df, stats_df, _ = continuous_backtest(commodity, data_mode, interval, strategy_name, strategy_params, capital, start, end)

### 绘制回测结果

In [ ]:
%matplotlib notebook
plt.figure(figsize=(10, 16))


balance_plot = plt.subplot(4, 1, 1)
balance_plot.set_title("Balance")
balance_plot.plot(stats_df["balance"].index, stats_df["balance"].values)

drawdown_plot = plt.subplot(4, 1, 2)
drawdown_plot.set_title("DrawdownPercent")
drawdown_plot.fill_between(stats_df["ddpercent"].index, stats_df['ddpercent'].values, facecolor='green', alpha=0.5)

pnl_plot = plt.subplot(4, 1, 3)
pnl_plot.set_title("Daily Pnl")
stats_df["net_pnl"].plot(legend=False)

distribution_plot = plt.subplot(4, 1, 4)
distribution_plot.set_title("Daily Pnl Distribution")
stats_df["net_pnl"].hist(bins=50)

plt.tight_layout() 

### 绘制成交点

In [ ]:
long_df = trade_df[trade_df.direction == 'long']
short_df = trade_df[trade_df.direction == 'short']

# 计算成交点x,y序列
symbol = comodity_to_vt_symbol(commodity, 'main')
price = load_data(symbol, interval, start, end)
price['id'] = range(len(price))

long_x = price.loc[long_df.index].id.values
long_y = long_df.price.values
short_x = price.loc[short_df.index].id.values
short_y = short_df.price.values

# 绘制成交点位
%matplotlib notebook
price2 = price.reset_index()
axe = price2.close.plot(figsize=(12, 6), color='#a8a8a8', zorder=10)
axe.scatter(long_x, long_y, color='r', marker=6, zorder=20)
axe.scatter(short_x, short_y, color='g', marker=7, zorder=30)
# plt.show()